<a href="https://colab.research.google.com/github/szambetti/INVEST_IO/blob/master/market_screener_reccom_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint
import numpy as np

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (121 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [41]:
# RACCOMANDAZIONI
url = 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news-broker-research/'
wd.set_window_size
wd.get(url)

# fai uno screenshot iniziale
wd.save_screenshot('initial_screenshot.png')

output = []

#loop sulle raccomandazioni dalla pagina principale
for i in wd.find_elements_by_css_selector('table#ALNI4.tabBody >* tr'):
  date = i.find_elements_by_css_selector('.pleft5.pright5.large30')[0].text
  reccom = i.find_elements_by_css_selector('.newsColCT.ptop3')[0].text
  link = i.find_elements_by_css_selector('.newsColCT.ptop3 > a')[0].get_attribute('href')
  output.append({'date': date, 'reccom' : reccom , 'link' : link})

In [42]:
def loop_reccom(j):
  for j 
  # RACCOMANDAZIONI
  url = 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news-broker-research'
  wd.set_window_size
  wd.get(url)

  # fai uno screenshot iniziale
  wd.save_screenshot('initial_screenshot.png')

  output = []

  #loop sulle raccomandazioni dalla pagina principale
  for i in wd.find_elements_by_css_selector('table#ALNI4.tabBody >* tr'):
    date = i.find_elements_by_css_selector('.pleft5.pright5.large30')[0].text
    reccom = i.find_elements_by_css_selector('.newsColCT.ptop3')[0].text
    link = i.find_elements_by_css_selector('.newsColCT.ptop3 > a')[0].get_attribute('href')
    output.append({'date': date, 'reccom' : reccom , 'link' : link})

  next_button = wd.find_elements_by_css_selector('.nPageEndTab')
  next_button.click()


[{'date': '08/05',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Kepler-Cheuvreux-reaffirms-its-Buy-rating-31061006/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Kepler Cheuvreux reaffirms its Buy rating'},
 {'date': '08/04',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Jefferies-reiterates-its-Buy-rating-31056906/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Jefferies reiterates its Buy rating'},
 {'date': '08/04',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Receives-a-Buy-rating-from-Jefferies-31053715/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Receives a Buy rating from Jefferies'},
 {'date': '08/04',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Gets-a-Buy-rating-from-JP-Morgan-31053569/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Gets a Buy rating from JP Morgan'},
 {'date': '07/31',
  'link': 'https:

In [ ]:
#ELENCO NEWS
#eventRowId_410946 > td.left.textNum.sentiment.noWrap
#wd.get('https://www.investing.com/economic-calendar/')

url = 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/'
import time
import numpy as np

from selenium import webdriver
wd.get(url)



all_rows = wd.find_elements_by_xpath('//*[@id="ALNI4"]/tbody/tr')
list_of_news=[]
for row in all_rows:
    
      td_1 = row.find_elements_by_css_selector("td")[0].text
      td_2 = row.find_elements_by_css_selector("td")[1].text
      list_of_news.append({'c': td_1,
                  'd': td_2})

#Extract Message for each user on a page //*[@id="eventRowId_410946"]/td[3]
#volatility = wd.find_element_by_xpath('//*[@id="eventRowId_410946"]/td[3]')[0]
#volatility_lvl = volatility.text
#print(volatility_lvl)

         c                                                  d
0   02:44p  Intesa secures full control in takeover battle...
1   01:10p  INTESA SANPAOLO : May Have Secured More Than 8...
2   12:51p  Intesa secures full control in takeover battle...
3   01:55a  INTESA SANPAOLO : Expects UBI's Takeover to Be...
4    07/27  Italy's Intesa Sees Takeover Bid For UBI Desti...
5    07/27  Italy's Consob says Intesa's bid for UBI exten...
6    07/27  Italy's Intesa says UBI bid to succeed as dead...
7    07/27  INTESA SANPAOLO : Shares in Ubi Banca fall as ...
8    07/27  &LDQUO;INIZIATIVA SOLLIEVO&RDQUO; : new 60 and...
9    07/24  INTESA SANPAOLO : Bid For UBI Reaches 32.662% ...
10   07/24  INTESA SANPAOLO : Italian bank Intesa expects ...
11   07/24  INTESA SANPAOLO : Exetra in support of Scotta ...
12   07/23  INTESA SANPAOLO : Italy's UBI rejects Intesa's...
13   07/23  INTESA SANPAOLO : Bid For UBI Reaches 26.406% ...
14   07/22  INTESA SANPAOLO : Take-up in Intesa's bid for ...
